In [0]:
import requests, json

base_url = "https://api.stackexchange.com/2.3/questions"
page = 1
while True:
    params = {
        "order": "desc",
        "sort": "creation",
        "site": "stackoverflow",
        "pagesize": 100,
        "page": page,
        "key": "rl_M1ZjooB6SNUAif5gAuSiS4Uzw" # TODO: store this in key vault later
    }
    resp = requests.get(base_url, params=params).json()
    if not resp.get("items"):
        break
    
    path = f"/mnt/so/bronze/questions_page={page}.json"
    dbutils.fs.put(path, json.dumps(resp), overwrite=True)
    
    if not ( resp.get("has_more", False) or resp.get("quota_remaining", 0) > 0 ):
        break
    page += 1


Wrote 78023 bytes.
Wrote 78148 bytes.
Wrote 79329 bytes.
Wrote 78471 bytes.
Wrote 78030 bytes.
Wrote 79024 bytes.
Wrote 79865 bytes.
Wrote 80391 bytes.
Wrote 79840 bytes.
Wrote 78380 bytes.
Wrote 77130 bytes.
Wrote 79577 bytes.
Wrote 78734 bytes.
Wrote 79918 bytes.
Wrote 78405 bytes.
Wrote 80197 bytes.
Wrote 78072 bytes.
Wrote 80955 bytes.
Wrote 78602 bytes.
Wrote 79809 bytes.
Wrote 77903 bytes.
Wrote 79456 bytes.
Wrote 80355 bytes.
Wrote 79831 bytes.
Wrote 79053 bytes.
Wrote 78542 bytes.
Wrote 78803 bytes.
Wrote 78487 bytes.
Wrote 80435 bytes.
Wrote 78027 bytes.
Wrote 79472 bytes.
Wrote 78902 bytes.
Wrote 80302 bytes.
Wrote 76719 bytes.
Wrote 77712 bytes.
Wrote 78512 bytes.
Wrote 78829 bytes.
Wrote 78285 bytes.
Wrote 79511 bytes.
Wrote 78933 bytes.
Wrote 77272 bytes.
Wrote 79269 bytes.
Wrote 77632 bytes.
Wrote 76966 bytes.
Wrote 77373 bytes.
Wrote 77870 bytes.
Wrote 77262 bytes.
Wrote 78070 bytes.
Wrote 76298 bytes.
Wrote 77957 bytes.
Wrote 77129 bytes.
Wrote 78801 bytes.
Wrote 77751 

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:132)
	at scala.Option.getOrElse(Option.scala:201)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:132)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:190)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:201)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

In [0]:
from pyspark.sql.functions import col, explode

df = spark.read.json("/mnt/so/bronze/*.json")

# Items array → rows
df_items = df.select(explode("items").alias("q"))

# Flatten minimal schema
df_flat = df_items.select(
    col("q.question_id").alias("id"),
    col("q.title"),
    col("q.view_count"),
    col("q.creation_date"),
    col("q.tags")
)

df_flat.write.format("delta").mode("overwrite").save("/mnt/so/silver/questions")


In [0]:
%sql
CREATE TABLE IF NOT EXISTS stackoverflow_questions
USING DELTA
LOCATION '/mnt/so/silver/questions';


In [0]:
%sql
SELECT * FROM stackoverflow_questions ORDER BY id DESC LIMIT 10;

id,title,view_count,creation_date,tags
79777115,Split-Plot Design Analysis using Base R,3,1759031968,List(anova)
79777107,When a projection of a variety is a variety,12,1759031264,List(math)
79777103,iText7 Pdfwriter failing,18,1759030796,"List(c#, .net, itext)"
79777101,Looking for advice on how to work with json files that sort data in atypical format,27,1759029945,"List(r, json)"
79777097,How to get room availability from opera OHIP APIs?,16,1759029534,"List(java, opera)"
79777096,Run Custom Tflite Model in Flutter | Didn't find op for builtin opcode 'CAST' version '5',13,1759029292,"List(flutter, runtime-error, tflite)"
79777090,GDB exiting my program when exit() shouldn't be invoked,14,1759028589,"List(gdb, reverse-engineering, elf)"
79777086,Consuming message from weblogic jms queue in wso2 mi,25,1759027609,"List(wso2, wso2-enterprise-integrator)"
79777083,Why does running a shader program trigger multiple OpenGL error?,27,1759026664,"List(c++, opengl, error-handling)"
79777082,Spring Boot MissingServletRequestPartException for 'occurrence' despite correct @RequestPart in List MultipartFile upload,22,1759026278,"List(spring, spring-boot, react-native)"


In [0]:
%sql
SELECT * FROM stackoverflow_questions ORDER BY id LIMIT 10;

id,title,view_count,creation_date,tags
77905667,Dynamic unicode message content,87,1706610950,"List(unicode, clickatell)"
77905674,Google Cloud Shell VM not starting,679,1706611069,"List(google-cloud-platform, google-cloud-shell)"
77905677,OpenModelica - All examples including EnergyPlus_9_6_0 fail,334,1706611079,"List(modelica, openmodelica, energyplus)"
77905682,Problem using a dynamic array as a member of a struct,86,1706611115,"List(arrays, c, struct, dynamic-arrays)"
77905685,How to use a header only library in C++ using cmake,251,1706611128,"List(c++, cmake)"
77905689,Excel: Charting graphs for different row numbers,32,1706611167,List(excel)
77905696,RecyclerView is not refreshed in real time when adding data,30,1706611229,"List(java, android, kotlin, android-recyclerview)"
77905699,Problems with minutes in timestamps when inserting data into InfluxDB via PowerShell,114,1706611239,"List(database, powershell, influxdb, influxql)"
77905706,Return to same ViewComponent from Controller action,52,1706611338,"List(c#, asp.net-core, razor, asp.net-core-mvc)"
77905707,Is the docs of the cnosdb wrong?,13,1706611339,List(cnosdb)


In [0]:
%sql
SELECT COUNT(*) FROM stackoverflow_questions;

count(1)
493700


In [0]:
files = dbutils.fs.ls("/mnt/so/bronze/")
print(sum([f.size for f in files])/(1024**3), 'gb')

0.3569591445848346 gb


In [0]:
%sql
select dayish, count(dayish) c from (select int(creation_date/100000) dayish from stackoverflow_questions) x group by dayish;

dayish,c
17420,363
17389,802
17437,762
17346,1058
17378,518
17396,424
17392,982
17525,404
17430,897
17554,141


In [0]:
from pyspark.sql.functions import avg
_sqldf.agg(avg('c')).collect()[0][0]


940.3809523809524